# Get mlrpt working with the saved data

In [1]:
library(tidyverse)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
library(lubridate)


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



In [3]:
library(assertthat)


Attaching package: ‘assertthat’

The following object is masked from ‘package:tibble’:

    has_name



In [4]:
paste("Running at ", Sys.time() %>% .POSIXct("GMT"), "GMT")

[1] "Running at  2019-09-15 15:19:43 GMT"

In [5]:
assert_that(length(list.files(pattern="satellite_data_*")) > 0,
            msg = "No satellite data files found on disk. Have you run `Meteor orbit data`?")

[1] TRUE

In [6]:
satellite_data_df <- readRDS("satellite_data_df.rds")

## Convert to satellite passes each day

Get the current time in UTC

In [7]:
current_time_utc <- Sys.time() %>% .POSIXct("GMT")

Make sure we have a future satellite pass in the data, then filter out any past passes.

In [8]:
assert_that(nrow(satellite_data_df %>% filter(startUTC > current_time_utc)) > 0,
            msg = "No future satellite data available")

[1] TRUE

In [9]:
satellite_data_df <- satellite_data_df %>% filter(startUTC > current_time_utc)

## Format systemd / mlrpt commands

For systemd, using `systemd-run` rather than the usual template based form since these are one-off non-recurring commands. The argument `--user` runs the command as the user who scheduled it. `--no-block` tells systemd to not synchronously wait for the unit start operation to finish. `--unit` gives the job a name. The argument `--on-calendar` schedules the start and needs to be one minute before the satellite pass to give systemd and mlrpt time to start up. The format is yyyy-mm-dd HH:MM in **local time**. Documentation at https://www.freedesktop.org/software/systemd/man/systemd-run.html

For `mlrpt` need arguments in the form of `mlrpt $startTime-$stopTime -t $duration`. Start and stop time are in the format HH:MM in **UTC**. Duration is how long the command needs to run (in **minutes**) from start to shut-down. Make it 3 minutes longer than the time of the satellite pass to give time for start-up, and at the end, decoding and saving images.

Times were already formatted and adjusted when creating the satellite pass dataframe and can be used directly in here.

In [10]:
paste0('systemd-run --user --no-block --unit=\"mlrpt_', satellite_data_df$startDate, "_",
       satellite_data_df$localStartTime,
       '\" --on-calendar=\"', satellite_data_df$startDate," ", satellite_data_df$localStartTime,
       '\" /usr/local/bin/mlrpt \"', satellite_data_df$startTime,"-", satellite_data_df$endTime,
       '\" -t ', satellite_data_df$duration)

[1] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-15_21:12\" --on-calendar=\"2019-09-15 21:12\" /usr/local/bin/mlrpt \"19:13-19:28\" -t 18"
 [2] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-16_08:29\" --on-calendar=\"2019-09-16 08:29\" /usr/local/bin/mlrpt \"06:30-06:45\" -t 18"
 [3] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-16_20:52\" --on-calendar=\"2019-09-16 20:52\" /usr/local/bin/mlrpt \"18:53-19:09\" -t 19"
 [4] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-17_08:08\" --on-calendar=\"2019-09-17 08:08\" /usr/local/bin/mlrpt \"06:09-06:25\" -t 19"
 [5] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-17_20:32\" --on-calendar=\"2019-09-17 20:32\" /usr/local/bin/mlrpt \"18:33-18:49\" -t 19"
 [6] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-18_07:49\" --on-calendar=\"2019-09-18 07:49\" /usr/local/bin/mlrpt \"05:50-06:05\" -t 18"
 [7] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-18_20:13\" --on-calendar=\"2019-09-18 20:13\" /usr/local/bin/mlrpt \"18:14-18:29\" -t 18"
 [8] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-19_07:29\" --on-calendar=\"2019-09-19 07:29\" /usr/local/bin/mlrpt \"05:30-05:45\" -t 18"
 [9] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-19_19:53\" --on-calendar=\"2019-09-19 19:53\" /usr/local/bin/mlrpt \"17:54-18:09\" -t 18"
[10] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-19_21:33\" --on-calendar=\"2019-09-19 21:33\" /usr/local/bin/mlrpt \"19:34-19:48\" -t 17"
[11] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-20_07:10\" --on-calendar=\"2019-09-20 07:10\" /usr/local/bin/mlrpt \"05:11-05:25\" -t 17"
[12] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-20_08:50\" --on-calendar=\"2019-09-20 08:50\" /usr/local/bin/mlrpt \"06:51-07:05\" -t 17"
[13] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-20_19:34\" --on-calendar=\"2019-09-20 19:34\" /usr/local/bin/mlrpt \"17:35-17:48\" -t 16"
[14] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-20_21:13\" --on-calendar=\"2019-09-20 21:13\" /usr/local/bin/mlrpt \"19:14-19:29\" -t 18"
[15] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-21_08:29\" --on-calendar=\"2019-09-21 08:29\" /usr/local/bin/mlrpt \"06:30-06:45\" -t 18"
[16] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-21_20:53\" --on-calendar=\"2019-09-21 20:53\" /usr/local/bin/mlrpt \"18:54-19:09\" -t 18"
[17] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-22_08:09\" --on-calendar=\"2019-09-22 08:09\" /usr/local/bin/mlrpt \"06:10-06:25\" -t 18"
[18] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-22_20:33\" --on-calendar=\"2019-09-22 20:33\" /usr/local/bin/mlrpt \"18:34-18:49\" -t 18"
[19] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-23_07:49\" --on-calendar=\"2019-09-23 07:49\" /usr/local/bin/mlrpt \"05:50-06:06\" -t 19"
[20] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-23_20:13\" --on-calendar=\"2019-09-23 20:13\" /usr/local/bin/mlrpt \"18:14-18:29\" -t 18"
[21] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-24_07:30\" --on-calendar=\"2019-09-24 07:30\" /usr/local/bin/mlrpt \"05:31-05:46\" -t 18"
[22] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-24_19:54\" --on-calendar=\"2019-09-24 19:54\" /usr/local/bin/mlrpt \"17:55-18:09\" -t 17"
[23] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-24_21:34\" --on-calendar=\"2019-09-24 21:34\" /usr/local/bin/mlrpt \"19:35-19:49\" -t 17"
[24] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-25_07:10\" --on-calendar=\"2019-09-25 07:10\" /usr/local/bin/mlrpt \"05:11-05:26\" -t 18"
[25] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-25_08:50\" --on-calendar=\"2019-09-25 08:50\" /usr/local/bin/mlrpt \"06:51-07:05\" -t 17"